In [2]:
import numpy as np

import pandas as pd
import geopandas as gpd

#import matplotlib.pyplot as plt
#import seaborn as sns

#import folium
import requests

import json

from random import randint

In [3]:
from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool, Select
from bokeh.plotting import figure
from bokeh.palettes import brewer
#from bokeh.sampledata.sample_geojson import geojson
from bokeh.layouts import widgetbox, row, column
from bokeh.io.doc import curdoc


from bokeh.tile_providers import CARTODBPOSITRON, get_provider

#output_notebook()

In [7]:
data = requests.get('https://data.gov.au/geoserver/vic-suburb-locality-boundaries-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_af33dd8c_0534_4e18_9245_fc64440f742e&outputFormat=json', headers={"Content-Type":"application/json"})
geodata = data.json()

with open('../data/raw/geodata.json', 'w', encoding='utf-8') as f:
    json.dump(geodata, f, indent=4)

In [29]:
gdf = gpd.read_file('../data/raw/geodata.json')
gdf.set_crs('epsg:4283')
# map tile providers provide only a few projections, so casting to that specific projection
# WGS84 (EPSG:4326) or Web Mercator (EPSG:3857) are usually available 
# source: https://kodu.ut.ee/~kmoch/geopython2020/L6/interactive-map-bokeh.html
gdf['geometry'] = gdf['geometry'].to_crs(epsg=3857) 
gdf.head()


,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry
0,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"MULTIPOLYGON (((15779040.302 -4173707.184, 157..."
1,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"MULTIPOLYGON (((16549732.035 -4494406.666, 165..."
2,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.3,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"MULTIPOLYGON (((15910102.922 -4576437.989, 159..."
3,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.4,6673,2011-08-31,None,VIC734,2018-08-03,None,DEPTFORD,None,None,G,None,2,"MULTIPOLYGON (((16455621.450 -4531505.374, 164..."
4,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.5,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"MULTIPOLYGON (((15727194.252 -4300425.394, 157..."


In [30]:
gdf['base'] = 100
gdf['1bedroom_pred'] = np.random.randint(75, 125, gdf.shape[0])
gdf['2bedroom_pred'] = np.random.randint(100, 150, gdf.shape[0])

gdf.head()

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry,base,1bedroom_pred,2bedroom_pred
0,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"MULTIPOLYGON (((15779040.302 -4173707.184, 157...",100,80,124
1,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"MULTIPOLYGON (((16549732.035 -4494406.666, 165...",100,108,124
2,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.3,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"MULTIPOLYGON (((15910102.922 -4576437.989, 159...",100,120,109
3,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.4,6673,2011-08-31,None,VIC734,2018-08-03,None,DEPTFORD,None,None,G,None,2,"MULTIPOLYGON (((16455621.450 -4531505.374, 164...",100,112,110
4,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.5,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"MULTIPOLYGON (((15727194.252 -4300425.394, 157...",100,81,115


In [31]:
#gdf['geometry'] = gdf['geometry'].simplify(0.05, preserve_topology=True) # if we want to reduce precision of each polygon to make it quicker to display, 0.05 is the reducing coeficitient or whateve

In [32]:
gdf['postal_code'] = gdf['loc_pid'].apply(lambda x: int(str(x).split('VIC')[1]))
gdf = gdf[gdf['postal_code'].isin(range(3000, 3792))]   # keep only greater Melbourne post indexes

In [34]:
# Set bokeh to save file
output_file("../plots/test.html", title="Reduced suburb plotting")

geo_source = GeoJSONDataSource(geojson=gdf.to_json())

# giving the basic view frame of the map
tile_provider = get_provider(CARTODBPOSITRON)
# range bounds supplied in web mercator coordinates
p = figure(x_range=(15600000, 16700000), y_range=(-5000000, -3900000),
           x_axis_type="mercator", y_axis_type="mercator",
           plot_height=600, plot_width=600)
p.add_tile(tile_provider)

# Define color palettes
palette = brewer['GnBu'][8]
# reverse order of colors so higher values have darker colors
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
# TODO, use those as min and max from data we plot
color_mapper = LinearColorMapper(palette=palette, low=75, high=125)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                     border_line_color=None, location=(0, 0), orientation='horizontal')
# major_label_overrides = tick_labels, could be used to set custom tickers, tick_labels is a dict value->display value


# Add patch renderer to figure.
suburbs = p.patches('xs', 'ys', source=geo_source,
                    # here comes the transform function
                    fill_color={'field': '1bedroom_pred',
                                'transform': color_mapper},
                    line_color='gray',
                    line_width=0.25,
                    fill_alpha=0.2)

# add tooltips as needed for other prediction values
tooltips = [('Suburb', '@vic_loca_2'),
            ('2025 prediction', '@1bedroom_pred')]
p.add_tools(HoverTool(renderers=[suburbs],
                      tooltips=tooltips))

show(p);  # this ; prevents vscode of showing output, output breaks things, todo to be fixef


Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "file:///home/toomas/ADS/generic-real-estate-consulting-project ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 


In [1]:
gdf.color = 'blue'
show(p);

NameError: name 'gdf' is not defined

In [9]:
from bokeh.io import show
from bokeh.models import CustomJS, RadioGroup

LABELS = ["Option 1", "Option 2", "Option 3"]

radio_group = RadioGroup(labels=LABELS, active=0)
radio_group.js_on_click(CustomJS(code="""
    console.log('radio_group: active=' + this.active, this.toString())
"""))

show(radio_group)


Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "file:///home/toomas/ADS/generic-real-estate-consulting-project ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 


In [10]:
# might be needed
output_file("../plots/test2.html", title="Testing suburb plotting")

palette = brewer['RdBu'][8]
palette = palette[::-1]

# Define the callback function: update_plot


def update_plot(attr, old, new):
    # The input cr is the criteria selected from the select box
    appartment_type = select.value

    # Update the plot based on the changed inputs
    p = make_plot(appartment_type)

    # Update the layout, clear the old document and display the new document
    layout = column(p, widgetbox(select))
    curdoc().clear()
    curdoc().add_root(layout)

    # Create a plotting function


def make_plot(field_name):
    # colorbar, todo make actual minmax
    min_range = 75
    max_range = 150

    color_mapper = LinearColorMapper(
        palette=palette, low=min_range, high=max_range)

    # Create color bar.
    color_bar = ColorBar(color_mapper=color_mapper,
                         border_line_color=None, location=(0, 0))

    # Create figure object.
    apt_type = gdf[field_name]

    p = figure(x_range=(15600000, 16700000), y_range=(-5000000, -3900000),
               x_axis_type="mercator", y_axis_type="mercator",
               plot_height=600, plot_width=600,
               title=field_name+' prediction for 2025')

    # Add patch renderer to figure.
    geo_source = GeoJSONDataSource(geojson=gdf.to_json())
    suburbs = p.patches('xs', 'ys', source=geo_source,
                        fill_color={'field': field_name,
                                    'transform': color_mapper},
                        line_color='black', line_width=0.25, fill_alpha=0.5)

    # Specify color bar layout.
    p.add_layout(color_bar, 'right')

    # Add the hover tool to the graph
    # add tooltips as needed for other prediction values
    tooltips = [('Suburb', '@vic_loca_2'),
                ('1 bedroom 2025 prediction', '@1bedroom_pred'),
                ('2 bedroom 2025 prediction', '@2bedroom_pred')]
    hover = HoverTool(renderers=[suburbs], tooltips=tooltips)

    p.add_tools(hover)
    return p


# Call the plotting function
input_field = '1bedroom_pred'
p = make_plot(input_field)


# Make a selection object: select
# add here other column names you want to plot
select = Select(title='Select apt type:', value='1bedroom_pred',
                options=['1bedroom_pred', '2bedroom_pred'])
select.on_change('value', update_plot)

# Make a column layout of select and plot, and add it to the current document
# Display the current document
layout = column(p, select)
curdoc().add_root(layout)


# output_notebook()
show(p)


Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "file:///home/toomas/ADS/generic-real-estate-consulting-project ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 
